In [54]:
import re
from functools import cache

In [41]:
with open("../input/day12-sample1.txt") as f:
    inp_sample1 = [line.rstrip() for line in f]

# with open("../input/01-sample2.txt") as f:
#     inp_sample2 = [line.rstrip() for line in f]

with open("../input/day12-input.txt") as f:
    inp = [line.rstrip() for line in f]

In [42]:
# inp = inp_sample1
# inp

In [43]:
def parse_line(line):
    springs, r = line.split(' ')
    damaged = [int(n) for n in r.split(',')]
    return (springs, damaged)

records = [parse_line(line) for line in inp]

In [44]:
def is_final(record):
    return "?" not in record[0]

is_final(records[0])

False

In [45]:
def replace_symbols(springs):
    return springs.replace('?', 'u').replace('.', 'o').replace('#', 'd')

def build_regex(damaged_groups):
    return "[uo]*" + ("[uo]+".join(f"[ud]{{{n}}}" for n in damaged_groups)) + "[uo]*"

def is_valid(record):
    text = replace_symbols(record[0])
    regex = build_regex(record[1])
    return re.fullmatch(regex, text)

In [46]:
def count_arrangements(record):
    candidates = deque()
    candidates.append(record)
    arrangements = 0

    while len(candidates) > 0:
        c = candidates.popleft()

        if is_valid(c):
            if is_final(c):
                arrangements += 1
            else:
                v1 = (c[0].replace('?', '.', 1), c[1])
                v2 = (c[0].replace('?', '#', 1), c[1])
                candidates.append(v1)
                candidates.append(v2)
    return arrangements

In [55]:
@cache
def count_arrangements2(springs, defects):
    if not defects:
        # still leftover damaged? -> invalid, otherwise -> valid
        if "#" in springs:
            return 0
        else:
            return 1
    if not springs:
        # still leftover defects? -> invalid
        if defects:
            return 0
        else:
            return 1
    
    if springs[0] == ".":
        return count_arrangements2(springs[1:], defects)
    elif springs[0] in "#?":
        result = 0
        
        if springs[0] == "?":
            result += count_arrangements2(springs[1:], defects)
        
        if (defects[0] <= len(springs)) and "." not in springs[:defects[0]]:
            # only if ? or all fixed
            if defects[0] == len(springs) or springs[defects[0]] != '#':
                result += count_arrangements2(springs[defects[0] + 1:], tuple(defects[1:]))
        return result
    else:
        return 0

In [65]:
# part 1
part1 = sum(count_arrangements2(r[0], tuple(r[1])) for r in records)
# sum(count_arrangements(r) for r in records)
part1

7251

In [57]:
def times5(record):
    return ("?".join([record[0]] * 5), record[1] * 5)

In [58]:
# part 2
records5 = [times5(record) for record in records]
# sum(count_arrangements(r) for r in records5)

In [64]:
cProfile.run('count_arrangements2(records5[1][0], tuple(records[1][1]))')

         137 function calls (75 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     63/1    0.000    0.000    0.000    0.000 3468598372.py:1(count_arrangements2)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
       71    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [66]:
part2 = sum(count_arrangements2(r[0], tuple(r[1])) for r in records5)
part2

2128386729962